In [ ]:
from functions import initialise, generate_image_new

def generate_image(payload=None, outfile=None, subfolder=None,test=None):
    if not (payload and outfile and subfolder and test):
        print("Please specify payload, outfile, subfolder,test")
        return False
    else:
        import os
        folder = "data/"+test+"/"+str(subfolder)
        if not os.path.exists(folder):
            os.makedirs(folder)
        response = requests.post(url=f'{url}/sdapi/v1/txt2img', json=payload)

        r = response.json()

        for i in r['images']:
            image = Image.open(io.BytesIO(base64.b64decode(i.split(",",1)[0])))

            png_payload = {
                "image": "data:image/png;base64," + i
            }
            response2 = requests.post(url=f'{url}/sdapi/v1/png-info', json=png_payload)

            pnginfo = PngImagePlugin.PngInfo()
            pnginfo.add_text("parameters", response2.json().get("info"))
            image.save(folder+'/'+str(outfile)+'.png', pnginfo=pnginfo)

In [ ]:
import json
import requests
import io
import base64
from PIL import Image, PngImagePlugin

url = "http://127.0.0.1:7860"
character = "linus_wd1_firefighter"
prompt = "linus_wd1, firefighter, winter, snow, by Michael & Inessa Garmash, night"
negative_prompt = "deformed, blurry, bad, disfigured, poor, mutation, mutated, ugly, horror, out of focus, depth of field, focal blur, worst quality, low quality, normal quality"
seed = 3413886288
seed = -1

samplers = [
"Euler a",
"Euler",
"LMS",
"Heun",
"DPM2",
"DPM2 a",
"DPM++ 2S a",
"DPM++ 2M",
"DPM fast",
"DPM adaptive",
"LMS Karras",
"DPM2 Karras",
"DPM2 a Karras",
"DPM++ 2S a Karras",
"DPM++ 2M Karras",
"DDIM",
"PLMS"
]

samplers = [
"Euler a",
"Euler",
"DPM++ 2S a Karras",
"DPM++ 2M Karras",
"DPM++ 2S a",
"DPM++ 2M",
]



payload, samplers = initialise(seed, prompt, negative_prompt, "10", samplers)


step_increase = 5
step = 10
while step <= 90:
    for sampler in samplers:
        if sampler == "LMS Karras" and step >= 90:
                print("Skipping LMS Karras, only noise from Step 90+")
        else:
            print(f"Sampler: {sampler} Step: {step}")
            payload["sampler_name"] = sampler
            payload["steps"] = step
            #generate_image(payload=payload, outfile=f"{sampler}_{step}",subfolder=character+"-"+str(payload['seed']),test="fullcheck")
            generate_image_new(url,payload=payload, outfile=f"{sampler}_{step}",subfolder=character+"-"+str(payload['seed']), save_file = True, save_file_folder="fullcheck", display_image_scale = 0.5, display_image = False)
    step+=step_increase
print("done")